In [1]:
using JSON
using DataFrames
using DataFramesMeta
using Dates
using ShiftedArrays
using OrderedCollections

In [2]:
f = "../share2/ndb_master_creation/output_iy/iycode_610406002_20240623_112942_859217.json"

"../share2/ndb_master_creation/output_iy/iycode_610406002_20240623_112942_859217.json"

In [3]:
d = JSON.parsefile(f)

Dict{String, Any} with 11 entries:
  "cat_generic"     => 1.0
  "dt_start"        => "2010-01-01"
  "nplcode"         => "2123014F2031"
  "dt_end"          => "2024-06-22"
  "dt"              => Dict{String, Any}("2012-05-16"=>Dict{String, Any}("point…
  "name"            => "アストニール錠１０　１０ｍｇ"
  "cat_neuro_break" => 0.0
  "drugcode"        => 610406002
  "cat_for_mental"  => 0.0
  "unit"            => "錠"
  "cat_bio"         => 0.0

In [4]:
# df = DataFrames.DataFrame(d["dt"])
l_dt = []
l_pt = []
l_st = []
for (k,v) in d["dt"]
    # print(k)
    push!(l_dt, k)
    push!(l_pt, v["point"])
    push!(l_st, v["tag"])
    # break
end

In [5]:
Dict("dt"=>l_dt)

Dict{String, Vector{Any}} with 1 entry:
  "dt" => ["2012-05-16", "2017-09-04", "2019-05-04", "2010-05-24", "2010-02-21"…

In [6]:
df = DataFrames.DataFrame(
    Dict("dt"=>l_dt,
         "point"=>l_pt,
         "tag"=>l_st)
)

d_dt = @chain df begin
    @transform(:dt_dt =Dates.Date.(:dt))
    @orderby(:dt_dt)
    @transform(:lag_tag =ShiftedArrays.lag(:tag, default = ""))
    @transform(:lag_dt =ShiftedArrays.lag(:dt, default = ""))
    @transform(:lag_point =ShiftedArrays.lag(:point, default = 0))
    @transform(:is_same_tag = :tag .== :lag_tag)
    @transform(:is_same_point = :point .== :lag_point)
    @transform(:is_same_two = :is_same_tag .& :is_same_point)
    @subset(:is_same_two .== false)
    @select(:dt_dt, :tag, :point)
end

d_dict = OrderedDict()

for r in eachrow(d_dt)
    # println(r)
    d_dict[r.dt_dt] = Dict("tag"=> r.tag, "point"=> r.point)
end
# d_dict =  Dict(d_dt.dt_dt .=> eachrow([d_dt.tag d_dt.Y]))
# d_1st = d_dt[!,"dt"]
d_dict

OrderedDict{Any, Any} with 3 entries:
  Date("2010-01-01") => Dict{String, Any}("point"=>14.1, "tag"=>"bdm")
  Date("2012-05-02") => Dict{String, Any}("point"=>13.1, "tag"=>"ws")
  Date("2013-04-02") => Dict{String, Any}("point"=>13.1, "tag"=>"aes")

In [7]:
# module rangedict
#     using DataFramesMeta
#     using JSON
#     using DataFrames
#     using Dates
#     using ShiftedArrays
#     using OrderedCollections
#     export RangeDict
    
    struct RangeDict
        d_1st
        d_dict
        d_meta
    end
    
    function gen_rd(f)
        d = JSON.parsefile(f)
        dts = pop!(d, "dt")
        l_dt = []
        l_pt = []
        l_st = []
        for (k,v) in dts
            # print(k)
            push!(l_dt, k)
            push!(l_pt, v["point"])
            push!(l_st, v["tag"])
            # break
        end
        df = DataFrames.DataFrame(
            Dict("dt"=>l_dt,
                 "point"=>l_pt,
                 "tag"=>l_st)
            )
        
        d_dt = @chain df begin
            @transform(:dt_dt =Dates.Date.(:dt))
            @orderby(:dt_dt)
            @transform(:lag_tag =ShiftedArrays.lag(:tag, default = ""))
            @transform(:lag_dt =ShiftedArrays.lag(:dt, default = ""))
            @transform(:lag_point =ShiftedArrays.lag(:point, default = 0))
            @transform(:is_same_tag = :tag .== :lag_tag)
            @transform(:is_same_point = :point .== :lag_point)
            @transform(:is_same_two = :is_same_tag .& :is_same_point)
            @subset(:is_same_two .== false)
            @select(:dt_dt, :tag, :point)
        end
        
        d_dict = OrderedDict()
        d_1st = Dict()
        for r in eachrow(d_dt)
            if length(keys(d_1st)) == 0
                d_1st = Dict("tag"=> r.tag, "point"=> r.point)
            end
            # println(r)
            d_dict[r.dt_dt] = Dict("tag"=> r.tag, "point"=> r.point)
        end
    rd_ = RangeDict(d_1st, d_dict, d)
    return rd_
    end

    function get_rd(self::RangeDict, dt)
        dt = Dates.Date(dt)
        res = self.d_1st
        for (k,v) in self.d_dict
            if dt < k
                break
            else
                res = copy(v)
            end
        end
        return res
    end

# end

get_rd (generic function with 1 method)

In [8]:
101 % 100

1

In [9]:
ts = "20240623_112942_859217"
d_fs = "../share2/ndb_master_creation/output_iy/"
fs = readdir("../share2/ndb_master_creation/output_iy/")
fs = [f for f in fs if occursin(ts, f)]
rd_iy = Dict{String,Any}()
for (i,f) in enumerate(fs)
    if i % 1000 == 0
        println(i)
    end
    pth = d_fs*f
    tag = f[8:16]
    # println(f)
    # println(tag)
    rd_iy[tag] = gen_rd(pth)
    # break
end

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


In [10]:
length(keys(rd_iy))

31892

In [13]:

get_rd(rd_iy["610406002"], "2014-03-12")


Dict{String, Any} with 2 entries:
  "point" => 13.1
  "tag"   => "aes"